In [2]:
!pip3 uninstall -y tira
!pip3 install git+https://github.com/tira-io/tira.git@development#\&subdirectory=python-client

Found existing installation: tira 0.0.135
Uninstalling tira-0.0.135:
  Successfully uninstalled tira-0.0.135
  Cloning https://github.com/tira-io/tira.git (to revision development) to /tmp/pip-req-build-wvvzxupc
  Running command git clone --filter=blob:none --quiet https://github.com/tira-io/tira.git /tmp/pip-req-build-wvvzxupc
  Running command git checkout -b development --track origin/development
  Switched to a new branch 'development'
  Branch 'development' set up to track remote branch 'development' from 'origin'.
  Resolved https://github.com/tira-io/tira.git to commit 287f2947754c2b06365321d2fcf058feefeea809
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tira: filename=tira-0.0.135-py3-none-any.whl size=100327 sha256=effb92151afb6dcae69e26e12b2d1bb34d81f2e0059f489ee96ccc6f1724463b
  Stored in directory: /tmp/pip-ephem-wheel-cache-_17jgmza/wheels/1f/36/7a/0f641175add91ee2

In [39]:
from tira.rest_api_client import Client
from tira.third_party_integrations import ir_datasets
import pandas as pd
from tqdm import tqdm
from ir_measures import *
from ir_measures import calc_aggregate

# do not truncate pandas columns and rows
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

tira = Client()

In [46]:
dataset_id = 'reneuir-2024/dl-top-1000-docs-20240701-training'
qrels = list(ir_datasets.load(dataset_id).qrels_iter())

APPROACHES = [
    'reneuir-2024/reneuir-baselines/plaid-x-retrieval', 'reneuir-2024/fschlatt/plaid-x-rank-zephyr', 'reneuir-2024/fschlatt/list-in-t5-re-rank-plaid-x',
    'reneuir-2024/fschlatt/monoelectra-large', 'reneuir-2024/fschlatt/monoelectra-base', 'reneuir-2024/fschlatt/rank-zephyr',
    'reneuir-2024/fschlatt/castorini-list-in-t5-150', 'reneuir-2024/fschlatt/sparse-cross-encoder-4-512', 'reneuir-2024/reneuir-baselines/llm-rankers-flan-t5-xl-top-100',
    'reneuir-2024/reneuir-baselines/llm-rankers-flan-t5-large-top-100', 'reneuir-2024/reneuir-baselines/llm-rankers-vicuna-13b-v1.5-top-100', 'reneuir-2024/reneuir-baselines/llm-rankers-flan-t5-xxl-top-100',
    'reneuir-2024/reneuir-baselines/llm-rankers-llama-2-7b-chat-top-100', 'reneuir-2024/reneuir-baselines/plaid-x-llm-rankers-flan-t5-xxl', 'reneuir-2024/reneuir-baselines/plaid-x-llm-rankers-flan-t5-xl',
    'reneuir-2024/reneuir-baselines/plaid-x-llm-rankers-vicuna-13b-v1.5', 'reneuir-2024/reneuir-baselines/plaid-x-llm-rankers-flan-t5-large', 'reneuir-2024/reneuir-baselines/plaid-x-llm-rankers-llama-2-7b-chat',
    'reneuir-2024/reneuir-baselines/anserini-bm25', 'reneuir-2024/reneuir-baselines/anserini-qld', 'reneuir-2024/reneuir-baselines/anserini-inl2',
    'reneuir-2024/reneuir-baselines/anserini-spl', 'reneuir-2024/reneuir-baselines/anserini-f2log', 'reneuir-2024/reneuir-baselines/anserini-f2exp',
    'reneuir-2024/tinyfsu/tiny-fsu-bert', 'reneuir-2024/tinyfsu/cross-encoder-ms-marco-TinyBERT-L-2', 'reneuir-2024/tinyfsu/tiny-bert-ranker-bm25',
    'reneuir-2024/tinyfsu/tiny-bert-ranker-l2-bm25', 'reneuir-2024/tinyfsu/tiny-bert-ranker-distillation', 'reneuir-2024/tinyfsu/tiny-bert-ranker-l-2-distillation',
    'reneuir-2024/naverlabseurope/Splade (re-ranker)', 'reneuir-2024/tu-dresden-03/bm25-flan-ul2-chain-of-thoughts-expansion-10', 'reneuir-2024/tu-dresden-03/bm25-flan-ul2-similar-queries-few-shot-5',
    'reneuir-2024/tu-dresden-03/bm25-flan-ul2-similar-queries-zero-shot-5', 'reneuir-2024/tu-dresden-03/bm25-llama2-chain-of-thoughts-10', 'reneuir-2024/tu-dresden-03/bm25-llama2-similar-queries-few-shot-5',
    'reneuir-2024/tu-dresden-03/bm25-llama2-similar-queries-zero-shot-5', 'reneuir-2024/tira-ir-starter/MonoT5 Base (tira-ir-starter-gygaggle)', 'reneuir-2024/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)',
    'reneuir-2024/tira-ir-starter/TASB msmarco-distilbert-base-cos (tira-ir-starter-beir)', 'reneuir-2024/tira-ir-starter/ANCE Base Dot (tira-ir-starter-beir)', 'reneuir-2024/tira-ir-starter/MonoT5 3b (tira-ir-starter-gygaggle)',
    'reneuir-2024/tira-ir-starter/MonoBERT Large (tira-ir-starter-gygaggle)', 'reneuir-2024/tira-ir-starter/MonoBERT Small (tira-ir-starter-gygaggle)', 'reneuir-2024/tira-ir-starter/SBERT multi-qa-mpnet-base-dot-v1 (tira-ir-starter-beir)',
    'reneuir-2024/tira-ir-starter/SBERT msmarco-distilbert-dot-v5 (tira-ir-starter-beir)', 'reneuir-2024/ows/pyterrier-anceretrieval', 'reneuir-2024/reneuir-baselines/mono-duo-t5-3b-top-50',
    'reneuir-2024/reneuir-baselines/mono-duo-t5-3b-top-25', 'reneuir-2024/reneuir-baselines/pyterrier-bm25', 'reneuir-2024/reneuir-baselines/pyterrier-pl2',
    'reneuir-2024/reneuir-baselines/pyterrier-dirichletlm', 'reneuir-2024/reneuir-baselines/pyterrier-dlh', 'reneuir-2024/reneuir-baselines/pyterrier-dph',
    'reneuir-2024/reneuir-baselines/pyterrier-ln-exp-b2', 'reneuir-2024/reneuir-baselines/fusion-monot5', 'reneuir-2024/reneuir-baselines/plaid-mono-t5',
    'reneuir-2024/fschlatt/set-encoder-base', 'reneuir-2024/fschlatt/set-encoder-large', 'reneuir-2024/fschlatt/fusion-t5-rank-zephyr',
    'reneuir-2024/reneuir-baselines/rankllama'
]

In [47]:
df_eval = []

for i in tqdm(APPROACHES):
    run = tira.pd.from_retriever_submission(i, dataset_id)

    # make the run compatible with ir_measures
    run['query_id'] = run['qid']
    run['doc_id'] = run['docno']
    
    calc_aggregate([nDCG@10], qrels, run)

    tmp = calc_aggregate([nDCG@10], qrels, run)
    df_eval += [{'approach': i, 'nDCG@10': calc_aggregate([nDCG@10], qrels, run)[nDCG@10]}]

df_eval = pd.DataFrame(df_eval)

df_eval.sort_values(by='nDCG@10', ascending=False).reset_index()[['approach', 'nDCG@10']]

100%|██████████| 61/61 [00:34<00:00,  1.78it/s]


,approach,nDCG@10
0,reneuir-2024/fschlatt/plaid-x-rank-zephyr,0.786125
1,reneuir-2024/fschlatt/fusion-t5-rank-zephyr,0.782695
2,reneuir-2024/reneuir-baselines/mono-duo-t5-3b-top-25,0.744482
3,reneuir-2024/reneuir-baselines/fusion-monot5,0.744038
4,reneuir-2024/reneuir-baselines/plaid-mono-t5,0.743741
5,reneuir-2024/fschlatt/list-in-t5-re-rank-plaid-x,0.743435
6,reneuir-2024/fschlatt/monoelectra-large,0.741231
7,reneuir-2024/tira-ir-starter/MonoT5 3b (tira-ir-starter-gygaggle),0.740981
8,reneuir-2024/reneuir-baselines/rankllama,0.734989
9,reneuir-2024/fschlatt/rank-zephyr,0.727169


In [7]:
df = []

for approach in APPROACHES:
    profiling = tira.profiling.from_submission(approach, 'dl-top-1000-docs-20240701-training')
    elapsed_time = profiling[-1]
    assert elapsed_time['key'] == 'elapsed_time'
    elapsed_time = elapsed_time['value']
    rss = [i['value'] for i in profiling if i['key'] == 'ps_vsz']
    gpu_memory_used = [int(i['value'].split()[0]) for i in profiling if i['key'] == 'gpu_memory_used']
    gpu_utilization = [int(i['value'].split()[0]) for i in profiling if i['key'] == 'gpu_utilization']
    cpu_utilization = [i['value'] for i in profiling if i['key'] == 'ps_cpu']
    df += [{
        'approach': approach.split('2024/')[1],
        'Elapsed Time': elapsed_time,
        'CPU Utilization (Max)': max(cpu_utilization),
        'RSS (MAX)': max(rss),
        'GPU Memory (Max)': max(gpu_memory_used),
        'GPU Utilization (Max)': max(gpu_utilization),
    }]

df = pd.DataFrame(df)
df.sort_values('Elapsed Time')

,approach,Elapsed Time,CPU Utilization (Max),RSS (MAX),GPU Memory (Max),GPU Utilization (Max)
0,tinyfsu/tiny-fsu-bert,64.0,135.7,17957372.0,250,23
1,tinyfsu/cross-encoder-ms-marco-TinyBERT-L-2,65.0,134.7,17953288.0,250,24
3,tinyfsu/tiny-bert-ranker-l2-bm25,77.0,210.7,17965852.0,250,23
2,tinyfsu/tiny-bert-ranker-bm25,100.0,135.7,17962184.0,250,14
9,tira-ir-starter/TASB msmarco-distilbert-base-c...,243.0,121.1,23648928.0,6380,100
4,tinyfsu/tiny-bert-ranker-distillation,343.0,104.4,28967756.0,2600,81
13,fschlatt/sparse-cross-encoder-4-512,360.0,143.2,394142648.0,3238,100
5,tinyfsu/tiny-bert-ranker-l-2-distillation,395.0,104.1,28966740.0,2600,72
10,tira-ir-starter/ANCE Base Dot (tira-ir-starter...,462.0,120.1,23146064.0,5880,100
8,tira-ir-starter/ColBERT Re-Rank (tira-ir-start...,910.0,113.4,27281316.0,2350,100
